## Preprocessing

In [3]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Determine the number of unique values in each column.
application_df.nunique()


APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts for binning
application_count = application_df['APPLICATION_TYPE'].value_counts()
application_count


T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_count[application_count < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df['CLASSIFICATION'].value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [9]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_count_gt1 = classification_count.loc[classification_count > 1]
classification_count_gt1.head()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_count[classification_count < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies_df = pd.get_dummies(application_df)
application_dummies_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [12]:
# Split our preprocessed data into our features and target arrays
X = application_dummies_df.drop('IS_SUCCESSFUL', axis=1).values
y = application_dummies_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [18]:
# 1st ATTEMPT - CHANING NODE VALUES FROM PREVIOUS NOTEBOOK (73% accuracy)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
total_input_features = len(X_train[0])
hidden_node1 = 23
hidden_node2 = 7


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node1, input_dim = total_input_features, activation = "relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node2, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 23)                1012      
                                                                 
 dense_4 (Dense)             (None, 7)                 168       
                                                                 
 dense_5 (Dense)             (None, 1)                 8         
                                                                 
Total params: 1188 (4.64 KB)
Trainable params: 1188 (4.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
# Compile the model (1st Attempt)
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
# Train the model (1st Attempt)
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5846 - accuracy: 0.7063
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5566 - accuracy: 0.7264
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5531 - accuracy: 0.7269
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5510 - accuracy: 0.7296
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5499 - accuracy: 0.7298
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5492 - accuracy: 0.7297
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5486 - accuracy: 0.7298
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5482 - accuracy: 0.7311
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5475 - accuracy: 0.7296
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5470 - accura

In [21]:
# Evaluate the model using the test data (1st Attempt)
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5565 - accuracy: 0.7299 - 290ms/epoch - 1ms/step
Loss: 0.5565442442893982, Accuracy: 0.729912519454956


In [14]:
# 2ND ATTEMPT - CHANING NODE VALUES AND CHANGING ACTIVATION FUNCTIONS (72.8%)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
total_input_features = len(X_train[0])
hidden_node1 = 50
hidden_node2 = 20


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node1, input_dim = total_input_features, activation = "relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node2, activation = "sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                2200      
                                                                 
 dense_1 (Dense)             (None, 20)                1020      
                                                                 
 dense_2 (Dense)             (None, 1)                 21        
                                                                 
Total params: 3241 (12.66 KB)
Trainable params: 3241 (12.66 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
# Compile the model (2nd Attempt)
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train the model (2nd Attempt)
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5790 - accuracy: 0.7144
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5570 - accuracy: 0.7278
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5536 - accuracy: 0.7298
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5516 - accuracy: 0.7301
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5504 - accuracy: 0.7305
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5492 - accuracy: 0.7306
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5485 - accuracy: 0.7313
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5479 - accuracy: 0.7314
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5476 - accuracy: 0.7306
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5470 - accura

In [17]:
# Evaluate the model using the test data (2nd Attempt)
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5552 - accuracy: 0.7277 - 446ms/epoch - 2ms/step
Loss: 0.5552440285682678, Accuracy: 0.7276967763900757


In [22]:
# 3RD ATTEMPT - ADDING ANOTHER NODE AND CHANGING ACTIVATION FUNCTIONS (72.7%)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
total_input_features = len(X_train[0])
hidden_node1 = 81
hidden_node2 = 33
hidden_node3 = 18

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node1, input_dim = total_input_features, activation = "relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node2, activation = "sigmoid"))

# Adding a third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node3, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 81)                3564      
                                                                 
 dense_7 (Dense)             (None, 33)                2706      
                                                                 
 dense_8 (Dense)             (None, 18)                612       
                                                                 
 dense_9 (Dense)             (None, 1)                 19        
                                                                 
Total params: 6901 (26.96 KB)
Trainable params: 6901 (26.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
# Compile the model (3rd Attempt)
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [24]:
# Train the model (3rd Attempt)
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.7382 - accuracy: 0.6999
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.7659 - accuracy: 0.7074
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5689 - accuracy: 0.7213
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5603 - accuracy: 0.7275
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5664 - accuracy: 0.7224
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5993 - accuracy: 0.7228
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5610 - accuracy: 0.7283
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5541 - accuracy: 0.7294
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5712 - accuracy: 0.7234
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5551 - accura

In [25]:
# Evaluate the model using the test data (3rd Attempt)
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6103 - accuracy: 0.7269 - 335ms/epoch - 1ms/step
Loss: 0.6103399395942688, Accuracy: 0.7268804907798767


In [33]:
# Optimizer: Keras Tuner
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=30,
        step=5), activation=activation, input_dim=total_input_features ))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=30,
            step=5),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [29]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 3.4 MB/s eta 0:00:00


In [34]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

Reloading Tuner from ./untitled_project/tuner0.json


In [35]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=100,validation_data=(X_test_scaled,y_test))

Trial 56 Complete [00h 00m 42s]
val_accuracy: 0.7281632423400879

Best val_accuracy So Far: 0.7338775396347046
Total elapsed time: 00h 17m 41s


In [36]:
# Get top 3 model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'relu', 'first_units': 21, 'num_layers': 3, 'units_0': 11, 'units_1': 16, 'units_2': 16, 'units_3': 1, 'units_4': 26, 'tuner/epochs': 7, 'tuner/initial_epoch': 0, 'tuner/bracket': 1, 'tuner/round': 0}
{'activation': 'tanh', 'first_units': 11, 'num_layers': 4, 'units_0': 6, 'units_1': 1, 'units_2': 1, 'units_3': 11, 'units_4': 16, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0014'}
{'activation': 'tanh', 'first_units': 26, 'num_layers': 1, 'units_0': 26, 'units_1': 26, 'units_2': 16, 'units_3': 26, 'units_4': 6, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0041'}


In [38]:
# Evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5538 - accuracy: 0.7339 - 383ms/epoch - 1ms/step
Loss: 0.5538204908370972, Accuracy: 0.7338775396347046
268/268 - 1s - loss: 0.5531 - accuracy: 0.7329 - 510ms/epoch - 2ms/step
Loss: 0.5530596971511841, Accuracy: 0.7329446077346802
268/268 - 0s - loss: 0.5536 - accuracy: 0.7329 - 468ms/epoch - 2ms/step
Loss: 0.5535991787910461, Accuracy: 0.7329446077346802


In [39]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")